In [1]:
import time
start = time.time()
cantidad_especies = 30
n_jobs = 4
verbose = 100
carpeta_data_base = "./data_real"
carpeta_fold_base = "./folds_real"

In [2]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, dump

def carpeta_data():
    return carpeta_data_base

def carpeta_fold():
    return carpeta_fold_base

def obtener_especies():
    query = "SELECT * FROM (SELECT CONVERT(@row_number:=@row_number + 1, UNSIGNED) AS orden, m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie, (SELECT @row_number:=0) AS rn ORDER BY m.id_especie) a ORDER BY 1 LIMIT " + str(cantidad_especies)
    return util_bd.resultados_query(query)
        
def generar_fasta_especie(row_especie):
    os.mkdir(carpeta_data() + "/clase_" + str(row_especie[0]))
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data() + "/clase_" + str(row_especie[0]) + "/lncRNA.fa")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data() + "/clase_" + str(row_especie[0]) + "/PCT.fa")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + str(row_especie[1]) + " ORDER BY f.cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data() + "/clase_" + str(row_especie[0]) + "/CDS.fa")

if os.path.isdir(carpeta_data()):
    shutil.rmtree(carpeta_data())
os.mkdir(carpeta_data())

%time dump(obtener_especies(), carpeta_data() + "/info_clases.bin")
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_fasta_especie)(row_especie) for row_especie in obtener_especies())

print("Proceso finalizado")

CPU times: user 70.7 ms, sys: 22.8 ms, total: 93.5 ms
Wall time: 753 ms
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   16.8s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   20.3s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   20.5s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:   40.3s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   40.3s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   40.4s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   55.3s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   56.2s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  2.2min

In [3]:
#Construir llave de folds
import os
import sys
sys.path.append('./libs')
import util_fasta
import hashlib
from sklearn.externals.joblib import dump, load
import importlib

importlib.reload(util_fasta) 

def folder_clase(num_clase):
    return carpeta_data() + "/clase_" + str(num_clase)

def archivo_clase(num_clase, tipo):
    return folder_clase(num_clase) + "/" + tipo + ".fa"

def obtener_num_clases():
    num_clases = 0
    if not os.path.isfile(carpeta_data() + "/num_clases.bin"):
        while os.path.isdir(folder_clase(num_clases + 1)):
            num_clases = num_clases + 1
        dump(num_clases, carpeta_data() + "/num_clases.bin")
    num_clases = load(carpeta_data() + "/num_clases.bin")
    return num_clases

def iterador_clases():
    return range(1, obtener_num_clases() + 1)

def obtener_primera_secuencia(num_clase):
    secuencias = util_fasta.leer_fasta(archivo_clase(num_clase, "lncRNA"), 1)
    return list(secuencias.keys())[0]

def obtener_todas_las_secuencias():
    return {num_clase : obtener_primera_secuencia(num_clase) for num_clase in iterador_clases()}

def generar_llaves_clases():
    secuencias = obtener_todas_las_secuencias()
    llaves = {num_clase : "" for num_clase in secuencias.keys()}
    llaves[0] = "" #llave cero corresponde a todo el universo
    for i_clase in iterador_clases():
        llaves[0] += secuencias[i_clase]
        for j_clase in iterador_clases():
            if (i_clase != j_clase):
                llaves[j_clase] += secuencias[i_clase]
    llaves[0] = hashlib.sha224(llaves[0].encode()).hexdigest()
    for i_clase in iterador_clases():
        llaves[i_clase] = hashlib.sha224(llaves[i_clase].encode()).hexdigest()
    dump(llaves, carpeta_data() + "/llaves_clases.bin")

def obtener_llaves_clases():
    return load(carpeta_data() + "/llaves_clases.bin")

%time generar_llaves_clases()

print("Proceso finalizado")

CPU times: user 5.74 ms, sys: 3.91 ms, total: 9.65 ms
Wall time: 259 ms
Proceso finalizado


In [4]:
#Armando folds (archivos fasta)
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, load

def cargar_llaves_clases():
    return load(carpeta_data() + "/llaves_clases.bin")

def carpeta_fold_clase(llave):
    return carpeta_fold() + "/fold_clase_" + str(llave)

def archivo_fold_clase(llave, tipoTrainTest, tipoRNA):
    return carpeta_fold_clase(llave) + "/" + tipoTrainTest + "/" + tipoRNA + ".fa"

def armar_fold_final(tipo):
    llave = obtener_llaves_clases()[0]
    if not os.path.isdir(carpeta_fold_clase(llave)):
        os.mkdir(carpeta_fold_clase(llave))
    if not os.path.isdir(carpeta_fold_clase(llave) + "/train"):
        os.mkdir(carpeta_fold_clase(llave) + "/train")
    with open(archivo_fold_clase(llave, "train", tipo), "w+") as outfile:
        for num_clase in iterador_clases():
            with open(archivo_clase(num_clase, tipo), "r") as infile:
                for inline in infile:
                    outfile.write(inline)
    
def armar_fold_clase(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    os.mkdir(carpeta_fold_clase(llave))
    os.mkdir(carpeta_fold_clase(llave) + "/train")
    for tipo in ["lncRNA", "PCT", "CDS"]:
        with open(archivo_fold_clase(llave, "train", tipo), "w+") as outfile:
            for j_num_clase in iterador_clases():
                if num_clase != j_num_clase:
                    with open(archivo_clase(j_num_clase, tipo)) as infile:
                        for inline in infile:
                            outfile.write(inline)
    os.mkdir(carpeta_fold_clase(llave) + "/test")
    for tipo in ["lncRNA", "PCT"]:
        with open(archivo_fold_clase(llave, "test", tipo), "w+") as outfile:
            with open(archivo_clase(num_clase, tipo)) as infile:
                for inline in infile:
                    outfile.write(inline)

if os.path.isdir(carpeta_fold()):
    shutil.rmtree(carpeta_fold())
os.mkdir(carpeta_fold())

%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(armar_fold_final)(tipo) for tipo in ["lncRNA", "PCT", "CDS"])
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(armar_fold_clase)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   10.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   10.9s finished
CPU times: user 18.1 ms, sys: 0 ns, total: 18.1 ms
Wall time: 10.9 s
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]:

In [5]:
#Generando modelos CPAT de los folds
import sys
sys.path.append('./libs')
import util_caracteristicas
import os
import importlib

importlib.reload(util_caracteristicas)

def carpeta_fold_cpat(llave):
    return carpeta_fold_clase(llave) + "/cpat"

def generar_cpat_fold(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    archivo_CDS = archivo_fold_clase(llave, "train", "CDS")
    carpeta_cpat = carpeta_fold_cpat(llave)
    if not os.path.isdir(carpeta_cpat):
        os.mkdir(carpeta_cpat)
    util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)
    
def generar_cpat_final():
    llave = obtener_llaves_clases()[0]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    archivo_CDS = archivo_fold_clase(llave, "train", "CDS")
    carpeta_cpat = carpeta_fold_cpat(llave)
    if not os.path.isdir(carpeta_cpat):
        os.mkdir(carpeta_cpat)
    util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)

def limpieza_archivos_CDS(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    os.remove(archivo_fold_clase(llave, "train", "CDS"))

%time generar_cpat_final()
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_cpat_fold)(num_clase) for num_clase in iterador_clases())
%time limpieza_archivos_CDS(0)
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(limpieza_archivos_CDS)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

CPU times: user 14.7 ms, sys: 8.31 ms, total: 23.1 ms
Wall time: 8min 57s
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 19.5min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed: 19.6min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed: 19.6min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed: 20.0min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 39.3min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed: 39.4min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed: 39.4min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed: 39.5min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed: 58.9min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 59.0min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed: 59.1min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed: 59.2min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed: 78.5min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed: 78.6m

In [6]:
#Ejecutar cpat y diamond sobre los folds
import os
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def ejecutar_cpat_fold(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    carpeta_cpat = carpeta_fold_cpat(llave)
    util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace(".fa", ".cpat"))
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".r")
    util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace(".fa", ".cpat"))
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".r")

def ejecutar_cpat_diamond_final():
    llave = obtener_llaves_clases()[0]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    diamond_db = "./Diamond_BD/uniprot-viridiplantae-reviewed.dmnd"
    carpeta_cpat = carpeta_fold_cpat(llave)
    util_caracteristicas.ejecutar_diamond(archivo_lncRNA, diamond_db, archivo_lncRNA.replace(".fa", ".dmnd"))
    util_caracteristicas.ejecutar_diamond(archivo_PCT, diamond_db, archivo_PCT.replace(".fa", ".dmnd"))
    util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace(".fa", ".cpat"))
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".r")
    util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace(".fa", ".cpat"))
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".r")

%time ejecutar_cpat_diamond_final()
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(ejecutar_cpat_fold)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

CPU times: user 59.7 ms, sys: 35.5 ms, total: 95.1 ms
Wall time: 42min 30s
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 16.9min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed: 17.0min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed: 17.1min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed: 17.3min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 34.0min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed: 34.2min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed: 34.2min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed: 34.5min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed: 51.0min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 51.0min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed: 51.3min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed: 51.4min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed: 68.1min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed: 68.2

In [7]:
#Generar archivo de features por cada fold
import os
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def archivo_features_clase(llave, tipoTrainTest, tipoRNA):
    return archivo_fold_clase(llave, tipoTrainTest, tipoRNA).replace(".fa", ".ft")

def generar_features_fold(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    features_base_lncRNA = archivo_features_clase(obtener_llaves_clases()[0], "train", "lncRNA")
    features_base_PCT = archivo_features_clase(obtener_llaves_clases()[0], "train", "PCT")
    for tipo in ["train", "test"]:
        archivo_lncRNA = archivo_fold_clase(llave, tipo, "lncRNA")
        archivo_PCT = archivo_fold_clase(llave, tipo, "PCT")
        archivo_cpat_lncRNA = archivo_fold_clase(llave, "train", "lncRNA").replace(".fa", ".cpat")
        archivo_cpat_PCT = archivo_fold_clase(llave, "train", "PCT").replace(".fa", ".cpat")
        salida_lncRNA = archivo_features_clase(llave, tipo, "lncRNA")
        salida_PCT = archivo_features_clase(llave, tipo, "PCT")
        util_caracteristicas.generar_features(archivo_lncRNA, features_base_lncRNA, archivo_cpat_lncRNA, salida_lncRNA)
        util_caracteristicas.generar_features(archivo_PCT, features_base_PCT, archivo_cpat_PCT, salida_PCT)
    
def generar_features_final():
    llave = obtener_llaves_clases()[0]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    salida_lncRNA = archivo_features_clase(llave, "train", "lncRNA")
    salida_PCT = archivo_features_clase(llave, "train", "PCT")
    util_caracteristicas.generar_features_base(archivo_lncRNA, archivo_lncRNA.replace(".fa", ".cpat"), archivo_lncRNA.replace(".fa", ".dmnd"), salida_lncRNA)
    util_caracteristicas.generar_features_base(archivo_PCT, archivo_PCT.replace(".fa", ".cpat"), archivo_PCT.replace(".fa", ".dmnd"), salida_PCT)

def limpieza_archivos_finales_fasta_ruta(llave):
    for tipo in ["train", "test"]:
        for tipoRNA in ["lncRNA", "PCT"]:
            if os.path.isfile(archivo_fold_clase(llave, tipo, tipoRNA)):
                os.remove(archivo_fold_clase(llave, tipo, tipoRNA))
            if os.path.isfile(archivo_fold_clase(llave, tipo, tipoRNA).replace(".fa", ".cpat")):
                os.remove(archivo_fold_clase(llave, tipo, tipoRNA).replace(".fa", ".cpat"))
    
def limpieza_archivos_finales_fasta(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    limpieza_archivos_finales_fasta_ruta(llave)

%time generar_features_final()
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_features_fold)(num_clase) for num_clase in iterador_clases())
%time limpieza_archivos_finales_fasta(0)
%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(limpieza_archivos_finales_fasta)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

CPU times: user 32.5 s, sys: 829 ms, total: 33.3 s
Wall time: 44.6 s
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.9min


/home/jose/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  5.7min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  6.9min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  7.1min
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:  7.3min
[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  8.5min
[Parallel(n_jobs=4)]: Do

In [8]:
#Generar modelo final
#import sys
#sys.path.append('./libs')
#import util_modelo_final
#import importlib
#import os

#importlib.reload(util_modelo_final)

#def generar_modelo_final_por_especie(id_especie):
#    util_modelo_final.reentrenar_modelo_final(id_especie)

#def generar_modelo_final():
#    util_modelo_final.generar_modelo_final(obtener_especies())
    
#%time generar_modelo_final()
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_modelo_final_por_especie)(id_especie) for id_especie in #obtener_especies())
    
#print("Proceso finalizado")
end = time.time()
print(end - start)
import datetime
print(datetime.datetime.now())

21079.664623975754
2019-05-11 04:51:24.652283
